In [1]:
import pandas as pd
data = pd.read_excel("../../../db/clean_orj_ttc.xlsx")
data.head()

,text,label
0,hgs i̇hlalli geçiş bilgileri sorgulama hizmeti...,ekonomi
1,yıl yüzde büyüyen helal gıda pazarı yıl discov...,ekonomi
2,i̇ş gyo i̇zmir temel attı i̇ş gayrimenkul yatı...,ekonomi
3,vergi indiren fransa bin istihdam bekliyor fra...,ekonomi
4,ericsson işçi çıkaracak i̇sveç merkezli ericss...,ekonomi


In [5]:
data = data.dropna()

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3600 non-null   object
 1   label   3600 non-null   object
dtypes: object(2)
memory usage: 56.4+ KB


In [7]:
import gensim
review_text = data.text.apply(gensim.utils.simple_preprocess)
review_text

0       [hgs, hlalli, geçiş, bilgileri, sorgulama, hiz...
1       [yıl, yüzde, büyüyen, helal, gıda, pazarı, yıl...
2       [gyo, zmir, temel, attı, gayrimenkul, yatırım,...
3       [vergi, indiren, fransa, bin, istihdam, bekliy...
4       [ericsson, işçi, çıkaracak, sveç, merkezli, er...
                              ...                        
3595    [giyilebilir, teknoloji, alanında, gün, yeni, ...
3596    [abd, bulunan, johns, hopkins, üniversitesi, y...
3597    [mars, gezegeninin, yüzeyinde, cam, katmanları...
3598    [samsung, note, serisine, ait, note, birlikte,...
3599    [sefer, bin, metreden, düşme, testine, maruz, ...
Name: text, Length: 3600, dtype: object

In [8]:
from gensim.models.doc2vec import TaggedDocument
class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc, tags=[self.labels_list[idx]])
 
docLabels = data.label.tolist()
data_text = review_text.tolist()
sentences = TaggedDocumentIterator(data_text, docLabels)

In [9]:
model = gensim.models.Doc2Vec(
    vector_size=100,
    window=10,
    min_count=2,
    alpha=0.025,
    workers=11,
    epochs=20
)

In [10]:
model.build_vocab(sentences)

In [11]:
model.epochs

20

In [12]:
model.train(sentences,total_examples=model.corpus_count,epochs=model.epochs)

In [13]:
model.save("d2v_orj_ttc.doc2vec")